In [1]:
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.streaming import StreamingContext
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler
from pyspark.ml.feature import StopWordsRemover, Word2Vec, RegexTokenizer, Tokenizer
from pyspark.ml.classification import LogisticRegression
from pyspark.sql import Row
from pyspark.streaming.kafka import KafkaUtils
import pyspark.sql.functions as f
import json
import re
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import *
import sys
from pyspark.sql.types import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import from_json
from pyspark.sql.functions import lit

import pickle
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from numpy import zeros
from keras.models import model_from_json

Using TensorFlow backend.


In [2]:
sc

<SparkContext master=yarn appName=PySparkShell>

In [3]:
spark

In [4]:
spark.sparkContext.getConf().getAll()

[('spark.eventLog.enabled', 'true'),
 ('spark.executor.instances', '3'),
 ('spark.sql.execution.arrow.enabled', 'true'),
 ('spark.driver.memory', '4g'),
 ('spark.executor.cores', '5'),
 ('spark.executor.extraJavaOptions', '-XX:+UseG1GC'),
 ('spark.driver.extraJavaOptions', '-XX:+UseG1GC'),
 ('spark.driver.host', 'kafka1'),
 ('spark.driver.port', '35313'),
 ('spark.serializer', 'org.apache.spark.serializer.KryoSerializer'),
 ('spark.yarn.archive', 'hdfs:///user/spark/conf/spark-libs.jar'),
 ('spark.app.id', 'application_1641116526368_0119'),
 ('spark.ui.proxyBase', '/proxy/application_1641116526368_0119'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.sql.shuffle.partitions', '200'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.filters',
  'org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter'),
 ('spark.kryoserializer.buffer.max', '2047'),
 ('spark.executor.memoryOverhead', '1g'),
 ('spark.driver.memoryOverhead', '1g'),
 ('spark.executor.memory', '4g'),
 ('spark

In [5]:
csi_pos_neg = spark.read.csv("hdfs:///user/spark/datafile/csiposneg.csv", header=True)

In [7]:
csi_pos_neg

DataFrame[_c0: string, text: string, label: string]

In [7]:
type(csi_pos_neg)

pyspark.sql.dataframe.DataFrame

In [8]:
csi_pos_neg.rdd.getNumPartitions()

5

In [6]:
csi_pos_neg.show(5)

+---+--------------------+-----+
|_c0|                text|label|
+---+--------------------+-----+
|  0|threatmeter hacke...|    1|
|  1|first android mal...|    1|
|  2|adobe fixes six c...|    1|
|  3| scienceporn  in ...|    1|
|  4|riskware hmoyfzb ...|    1|
+---+--------------------+-----+
only showing top 5 rows



In [10]:
dfdf = csi_pos_neg.toPandas()
test_df = dfdf[dfdf['text'].str.contains('baseboard')]

/root/spark/python/pyspark/sql/dataframe.py:2151: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true, but has reached the error below and can not continue. Note that 'spark.sql.execution.arrow.fallback.enabled' does not have an effect on failures in the middle of computation.
  'MultiIndex' object has no attribute 'labels'
  warnings.warn(msg)


AttributeError: 'MultiIndex' object has no attribute 'labels'

In [8]:
test_df = spark.createDataFrame(test_df)

/root/spark/python/pyspark/sql/session.py:714: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  PyArrow >= 0.8.0 must be installed; however, it was not found.
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
  warnings.warn(msg)


In [83]:
test_df.show(truncate=False)

+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|_c0  |text                                                                                                                                                                                                                                                            |label|
+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|59263|cve out of bound read in intelr baseboard management controller firmware may allow an unauthenticated user to potentially enable information disclosure via network access  cgibincv

In [7]:
csi_pos_neg_test = spark.read.csv("hdfs:///user/spark/datafile/csiposnegtest.csv", header=True)

In [9]:
csi_pos_neg.groupBy('label').count().orderBy('count').show()

+-----+-----+
|label|count|
+-----+-----+
|    1|99756|
|    0|99756|
+-----+-----+



In [9]:
csi_pos_neg_test.groupBy('label').count().orderBy('count').show()

+-----+-----+
|label|count|
+-----+-----+
|    0|24939|
|    1|24939|
+-----+-----+



In [7]:
csi_pos_neg = csi_pos_neg.drop('_c0')

In [44]:
csi_pos_neg_test = csi_pos_neg_test.drop('_c0')

In [45]:
qq = csi_pos_neg_test.collect()[:24939]
qq = spark.createDataFrame(qq)

In [46]:
qq.groupBy('label').count().orderBy('count').show()

+-----+-----+
|label|count|
+-----+-----+
|    1|24939|
+-----+-----+



In [78]:
qq

DataFrame[text: string, label: string]

In [6]:
t1 = csi_pos_neg.collect()[:24939]
t2 = csi_pos_neg.collect()[24939:49878]
t3 = csi_pos_neg.collect()[49878:74817]
t4 = csi_pos_neg.collect()[74817:99756]

t5 = csi_pos_neg.collect()[99756:124695]
t6 = csi_pos_neg.collect()[124695:149634]
t7 = csi_pos_neg.collect()[149634:174573]
t8 = csi_pos_neg.collect()[174573:199512]

In [7]:
t1

[Row(_c0='0', text='threatmeter hacked emails of san francisco muni rail system hacker reveals clues about identity and tactics kr  fin ', label='1'),
 Row(_c0='1', text='first android malware targeting pcs uncovered  fbxtynm ', label='1'),
 Row(_c0='2', text='adobe fixes six code execution bugs in flash  mokgxr ', label='1'),
 Row(_c0='3', text=' scienceporn  in a vacuum i guess', label='1'),
 Row(_c0='4', text='riskware hmoyfzb  fbjaz ', label='1'),
 Row(_c0='5', text='rt exploithunt  wzemae   data protection in fusion itsec  databreach  news   wzenei  ', label='1'),
 Row(_c0='6', text='webroot  spamvertised you have  lost message on facebook campaign leads to pharmaceutical scams  xunah ', label='1'),
 Row(_c0='7', text=' cybersecurity  hacker  privacy and big data   rmge ', label='1'),
 Row(_c0='8', text='postsnag gingcredsfromlockedmachines  ', label='1'),
 Row(_c0='9', text='ddos  t  mvrzcc ', label='1'),
 Row(_c0='10', text='top  actions for runtime application protection  fbodt

In [8]:
t1 = spark.createDataFrame(t1)
t2 = spark.createDataFrame(t2)
t3 = spark.createDataFrame(t3)
t4 = spark.createDataFrame(t4)

t5 = spark.createDataFrame(t5)
t6 = spark.createDataFrame(t6)
t7 = spark.createDataFrame(t7)
t8 = spark.createDataFrame(t8)

In [9]:
first_train=test_df.union(t1).union(t5) #word_counts_1 t1
first_test=t2.union(t6)
# second_train=first_train.union(first_test) #word_counts_1 t1, t2
# second_test=t3.union(t7)
# third_train=second_train.union(second_test)
# third_test=t4.union(t8)
# fourth_train=third_train.union(third_test)
# #fourth_test

NameError: name 'test_df' is not defined

In [11]:
first_test = first_train.union(first_test)

In [11]:
a=t1.union(t2)
b=t5.union(t6)
second_train = a.union(b)

In [12]:
second_test=t3.union(t7)

In [46]:
second_test = second_train.union(second_test)

In [12]:
a=t1.union(t2).union(t3)
b=t5.union(t6).union(t7)
third_train = a.union(b)

In [45]:
third_test = t4.union(t8)
third_test = third_train.union(third_test)

In [46]:
third_test.count()

199512

In [12]:
a=test_df.union(t1).union(t2).union(t3).union(t4)
b=t5.union(t6).union(t7).union(t8)
fourth_train = a.union(b)

In [13]:
fourth_train.count()

199515

In [51]:
t8 = t8.drop('_c0')

In [52]:
ee = qq.union(t8)

In [53]:
ee.groupBy('label').count().orderBy('count').show()

+-----+-----+
|label|count|
+-----+-----+
|    1|24939|
|    0|24939|
+-----+-----+



In [56]:
fourth_train = fourth_train.drop('_c0')

In [57]:
fourth_test=fourth_train.union(ee)

In [13]:
second_train.show(5)

+--------------------+-----+
|                text|label|
+--------------------+-----+
|threatmeter hacke...|    1|
|first android mal...|    1|
|adobe fixes six c...|    1|
| scienceporn  in ...|    1|
|riskware hmoyfzb ...|    1|
+--------------------+-----+
only showing top 5 rows



In [60]:
first_train.show(5)

+--------------------+-----+
|                text|label|
+--------------------+-----+
|threatmeter hacke...|    1|
|first android mal...|    1|
|adobe fixes six c...|    1|
| scienceporn  in ...|    1|
|riskware hmoyfzb ...|    1|
+--------------------+-----+
only showing top 5 rows



In [61]:
first_test.show(5)

+--------------------+-----+
|                text|label|
+--------------------+-----+
|threatmeter hacke...|    1|
|first android mal...|    1|
|adobe fixes six c...|    1|
| scienceporn  in ...|    1|
|riskware hmoyfzb ...|    1|
+--------------------+-----+
only showing top 5 rows



In [16]:
fourth_train.groupBy('label').count().orderBy('count').show()

+-----+-----+
|label|count|
+-----+-----+
|    0|99756|
|    1|99756|
+-----+-----+



In [6]:
first_test.groupBy('label').count().orderBy('count').show()

NameError: name 'first_test' is not defined

In [13]:
with open('/root/word_counts1.json') as json_file:
    json_data = json.load(json_file)

word_counts_1 = {}
for i in json_data:
    word_counts_1.update({i[0]:i[1]})

In [14]:
word_counts_1

{'allow': 105727,
 'attacker': 90501,
 'via': 82323,
 'vulnerability': 80391,
 'remote': 78973,
 'arbitrary': 52410,
 'cve': 39210,
 'user': 36471,
 'execute': 33511,
 'file': 32873,
 'code': 30863,
 'service': 29505,
 'script': 28010,
 'cause': 27498,
 'parameter': 25566,
 'server': 25181,
 'denial': 24858,
 'craft': 23383,
 'version': 22915,
 'web': 21729,
 'access': 20831,
 'earlier': 20587,
 'vector': 20532,
 'unspecified': 20528,
 'information': 18209,
 'windows': 17445,
 'affect': 16799,
 'aka': 15656,
 'x': 15573,
 'sql': 15292,
 'privilege': 15242,
 'local': 15007,
 'could': 14985,
 'command': 14982,
 'oracle': 14765,
 'application': 14611,
 'attack': 14514,
 'crosssite': 14312,
 'memory': 14155,
 'function': 14089,
 'overflow': 13750,
 'issue': 13572,
 'request': 13418,
 'use': 13171,
 'buffer': 13046,
 'data': 12762,
 'html': 12548,
 'obtain': 12177,
 'inject': 11741,
 'crash': 10967,
 'multiple': 10759,
 'system': 10717,
 'authenticate': 10656,
 'id': 10489,
 'sp': 10488,
 '

In [15]:
word_counts_1 = sc.broadcast(word_counts_1)

In [16]:
with open('/root/word_counts2.json') as json_file:
    json_data = json.load(json_file)

word_counts_2 = {}
for i in json_data:
    word_counts_2.update({i[0]:i[1]})
word_counts_2

{'like': 81920,
 'im': 74424,
 'get': 69810,
 'dont': 56228,
 'one': 53583,
 'people': 51475,
 'love': 49799,
 'say': 47532,
 'amp': 43535,
 'make': 42948,
 'know': 41443,
 'go': 38824,
 'see': 36548,
 'want': 35744,
 'good': 33602,
 'time': 33174,
 'need': 32592,
 'rt': 30873,
 'think': 27942,
 'day': 27370,
 'look': 27019,
 'new': 26230,
 'today': 25902,
 'really': 25633,
 'us': 25067,
 'cant': 24346,
 'never': 24085,
 'follow': 23866,
 'back': 23639,
 'let': 23283,
 'take': 22517,
 'please': 22147,
 'would': 21752,
 'still': 21303,
 'right': 20703,
 'someone': 20625,
 'happy': 20330,
 'life': 20204,
 'come': 20049,
 'youre': 19980,
 'tweet': 19948,
 'much': 19676,
 'trump': 19574,
 'thank': 19534,
 'best': 18467,
 'first': 18292,
 'thats': 18255,
 'friend': 18089,
 'give': 17781,
 'girl': 17582,
 'tell': 17505,
 'may': 17339,
 'even': 17303,
 'way': 16962,
 'video': 16777,
 'yall': 16721,
 'man': 16474,
 'shit': 16438,
 'going': 16289,
 'try': 16191,
 'everyone': 16037,
 'black': 15

In [17]:
word_counts_2 = sc.broadcast(word_counts_2)

In [18]:
# get_tokens 함수과 같은 용도
stage_1 = RegexTokenizer(inputCol= 'text', outputCol='pos_t', pattern= '\\W')

In [21]:
r1 = stage_1.transform(second_train)
r1.show(5)

+---+--------------------+-----+--------------------+
|_c0|                text|label|               pos_t|
+---+--------------------+-----+--------------------+
|  0|threatmeter hacke...|    1|[threatmeter, hac...|
|  1|first android mal...|    1|[first, android, ...|
|  2|adobe fixes six c...|    1|[adobe, fixes, si...|
|  3| scienceporn  in ...|    1|[scienceporn, in,...|
|  4|riskware hmoyfzb ...|    1|[riskware, hmoyfz...|
+---+--------------------+-----+--------------------+
only showing top 5 rows



In [20]:
# r1.select('text','label').rdd.map(lambda x: x).reduceByKey(lambda a,b: b+a).toDF(['key','value'])

In [22]:
# get_lemma 함수와 같은 용도
stage_2 = StopWordsRemover(inputCol= stage_1.getOutputCol(), outputCol= 'filtered_words')

In [23]:
r2 = stage_2.transform(r1)
r2.show(5)

+---+--------------------+-----+--------------------+--------------------+
|_c0|                text|label|               pos_t|      filtered_words|
+---+--------------------+-----+--------------------+--------------------+
|  0|threatmeter hacke...|    1|[threatmeter, hac...|[threatmeter, hac...|
|  1|first android mal...|    1|[first, android, ...|[first, android, ...|
|  2|adobe fixes six c...|    1|[adobe, fixes, si...|[adobe, fixes, si...|
|  3| scienceporn  in ...|    1|[scienceporn, in,...|[scienceporn, vac...|
|  4|riskware hmoyfzb ...|    1|[riskware, hmoyfz...|[riskware, hmoyfz...|
+---+--------------------+-----+--------------------+--------------------+
only showing top 5 rows



In [24]:
r2.show(1, truncate=False)

+---+--------------------------------------------------------------------------------------------------------------------+-----+-------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------+
|_c0|text                                                                                                                |label|pos_t                                                                                                                                |filtered_words                                                                                                       |
+---+--------------------------------------------------------------------------------------------------------------------+-----+--------------------------------------------------------------------------------------------------------------

In [25]:
len()

TypeError: len() takes exactly one argument (0 given)

In [26]:
r2.rdd.getNumPartitions()

652

In [27]:
r2.count()

99756

In [28]:
r2

DataFrame[_c0: string, text: string, label: string, pos_t: array<string>, filtered_words: array<string>]

In [29]:
class BroadcastWrapper(object):
    def __init__(self, data, token_list):
        self.broadcast_var = sc.broadcast(data)
#         self.last_updated_time = datetime.now()
        self.token_list = token_list
    
#     def is_should_be_updated(self, data):
#         cur_time = datetime.now()
#         diff_sec = (cur_time - self.last_updated_time).total_seconds()
#         return self.broadcast_var is None or diff_sec> 1
    
    def update_and_get_data(self, spark):
        a = self.broadcast_var.value
        self.broadcast_var.unpersist()
        for i in self.token_list:
            for j in i:
                if j not in a.keys():
                    a[j] = 1
                else:
                    a[j] += 1
        new_data = a
        self.broadcast_var = spark.broadcast(new_data)
#         self.last_updated_time = datetime.now()
#         return len(self.token_list)
        return self.broadcast_var

# Train용

In [30]:
import numpy as np
import pandas as pd
from pyspark import SparkContext
from pyspark.sql import SQLContext
df = pd.DataFrame(np.array(["a","b","c","d","e"]), columns=['row1'])
sparkdf = sqlContext.createDataFrame(df, samplingRatio=0.1)

In [31]:
sparkdf.show(5)

+----+
|row1|
+----+
|   a|
|   b|
|   c|
|   d|
|   e|
+----+



In [32]:
from pyspark.sql import Row
from pyspark.sql import types   

In [48]:
n = sparkdf.rdd.getNumPartitions()
n

5

In [122]:
sparkdf.select('row1').rdd.collect()

[Row(row1='a'), Row(row1='b'), Row(row1='c'), Row(row1='d'), Row(row1='e')]

In [123]:
type(sparkdf.rdd.collect())

list

In [138]:
old_array = [str(row['row1']) for row in sparkdf.select('row1').collect()]

In [139]:
old_array

['a', 'b', 'c', 'd', 'e']

In [46]:
import numpy as np
new_col = np.array([[1,2,3], [2,2,2], [3,3,3], [4,4,4], [5,4]])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [49]:
new_col = sc.parallelize(new_col, n).map(int)

In [50]:
new_col

PythonRDD[174] at RDD at PythonRDD.scala:53

In [51]:
new_array = new_col.collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 4 in stage 22.0 failed 4 times, most recent failure: Lost task 4.3 in stage 22.0 (TID 3331, kafka1, executor 1): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/root/spark/python/pyspark/worker.py", line 377, in main
    process()
  File "/root/spark/python/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/root/spark/python/pyspark/serializers.py", line 400, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/root/spark/python/pyspark/util.py", line 99, in wrapper
    return f(*args, **kwargs)
TypeError: int() argument must be a string, a bytes-like object or a number, not 'list'

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:45

In [170]:
zip_array = list(zip(old_array, new_col.tolist()))

In [171]:
zip_array

[('a', [1, 2, 3], [1, 2, 3], [1, 2, 3]),
 ('b', [2, 2, 2], [2, 2, 2], [2, 2, 2]),
 ('c', [3, 3, 3], [3, 3, 3], [3, 3, 3]),
 ('d', [4, 4, 4], [4, 4, 4], [4, 4, 4]),
 ('e', [5, 4], [5, 4], [5, 4])]

In [143]:
print(new_col.collect())

[1, 2, 3, 4, 5]


In [144]:
rdd_new = old_array.rdd.zip(new_col)

AttributeError: 'list' object has no attribute 'rdd'

In [129]:
rdd_new.collect()

[(Row(row1='a'), 1),
 (Row(row1='b'), 2),
 (Row(row1='c'), 3),
 (Row(row1='d'), 4),
 (Row(row1='e'), 5)]

In [162]:
transaction_detail_schema = StructType() \
    .add("row1", StringType()) \
    .add("row2", ArrayType(IntegerType()))

fdf = spark.createDataFrame(zip_array, transaction_detail_schema)

In [163]:
fdf.show()

+----+---------+
|row1|     row2|
+----+---------+
|   a|[1, 2, 3]|
|   b|[2, 2, 2]|
|   c|[3, 3, 3]|
|   d|[4, 4, 4]|
|   e|   [5, 4]|
+----+---------+



In [164]:
fdf.rdd.getNumPartitions()

18

In [132]:
input_data = [("Uttar Pradesh",[1,2,3]),
      ("Maharashtra",[1,2,3]),
      ("Tamil Nadu",[1,2,3]),
      ("Karnataka",[1,2,3]),
      ("Kerala",[1,2,3])]
df = sc.parallelize(input_data)

# creating the dataframe using createDataFrame function
df = spark.createDataFrame(df,transaction_detail_schema)
# printing schema of the dataframe and showing the dataframe
df.printSchema()
df.show()

root
 |-- row1: string (nullable = true)
 |-- row2: array (nullable = true)
 |    |-- element: integer (containsNull = true)

+-------------+---------+
|         row1|     row2|
+-------------+---------+
|Uttar Pradesh|[1, 2, 3]|
|  Maharashtra|[1, 2, 3]|
|   Tamil Nadu|[1, 2, 3]|
|    Karnataka|[1, 2, 3]|
|       Kerala|[1, 2, 3]|
+-------------+---------+



In [134]:
df2= df.collect()
df2

[Row(row1='Uttar Pradesh', row2=[1, 2, 3]),
 Row(row1='Maharashtra', row2=[1, 2, 3]),
 Row(row1='Tamil Nadu', row2=[1, 2, 3]),
 Row(row1='Karnataka', row2=[1, 2, 3]),
 Row(row1='Kerala', row2=[1, 2, 3])]

In [42]:
from pyspark.ml import Transformer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
from typing import Dict
from pyspark.sql import DataFrame
from pyspark.sql import Row
from pyspark.sql import types   

class TextToSequence(Transformer):
    w1 = dict()
    w2 = dict()
    pos_t = Tokenizer(lower=False)
    neg_t = Tokenizer(lower=False)
    pos_vocab = []
    neg_vocab = []
    
    def __init__(self, w1: Dict[str, int], w2:Dict[str, int]):
        super(TextToSequence, self).__init__()
        self.w1 = w1
        self.w2 = w2
        
    
    def _transform(self, df: DataFrame):
        result_pdf = df.select("filtered_words").toPandas()
        pos_df = result_pdf[3:99759] # 24939 99756
        neg_df = result_pdf[99759:]
        
        broadcast_wrapper1 = BroadcastWrapper(self.w1, pos_df["filtered_words"])
        ww1 = broadcast_wrapper1.update_and_get_data(sc).value
        
        broadcast_wrapper2 = BroadcastWrapper(self.w2, neg_df["filtered_words"])
        ww2 = broadcast_wrapper2.update_and_get_data(sc).value
        
        www1 = {k: v for k, v in sorted(ww1.items(), key=lambda item: item[1], reverse=True)}
        www2 = {k: v for k, v in sorted(ww2.items(), key=lambda item: item[1], reverse=True)}
        print(www1['allow'])
        print(www2['like'])
        aa = [w for i, w in enumerate(www1) if i < 5000]
        bb = [w for i, w in enumerate(www2) if i < 5000]
        
        self.pos_t.fit_on_texts(aa)
        self.neg_t.fit_on_texts(bb)
#         encoded_docs_pos = self.pos_t.texts_to_sequences(df.select("filtered_words1").toPandas())
#         encoded_docs_neg = self.neg_t.texts_to_sequences(df.select("filtered_words1").toPandas())
        encoded_docs_pos = self.pos_t.texts_to_sequences(result_pdf["filtered_words"])
        encoded_docs_neg = self.neg_t.texts_to_sequences(result_pdf["filtered_words"])
        
        X_p = pad_sequences(encoded_docs_pos, maxlen=100, padding='post').tolist()
        X_n = pad_sequences(encoded_docs_neg, maxlen=100, padding='post').tolist()
        
        text_array = [str(row['text']) for row in df.select('text').collect()]
        label_array = [int(row['label']) for row in df.select('label').collect()]

        
        zip_array = list(zip(text_array, label_array, X_p, X_n))
        rdd = sc.parallelize(zip_array, numSlices=306)
        fdf = rdd.toDF(['text','label','feature1','feature2'])
        
#         rowRDD = rdd.map(lambda x: Row(x[0], x[1], x[2], x[3]))



#         transaction_detail_schema = StructType() \
#             .add("text", StringType()) \
#             .add("label", IntegerType()) \
#             .add("feature1", ArrayType(IntegerType())) \
#             .add("feature2", ArrayType(IntegerType()))
        
#         fdf = spark.createDataFrame(zip_array, transaction_detail_schema)
        print(type(fdf))

    
    
    
    
    
    
    
    
    
#         cols = ['feature1','feature2']
#         def feature_add(x) :
#             tmp = []
#             tmp.append(x.values)
#             return tmp[0].tolist()
        
#         df.insert(0, 'features', df[cols].apply(feature_add, axis=1))
        
#         fdf = spark.createDataFrame(df)
        return fdf, www1, aa

In [43]:
# USE THE TRANSFORMER WITHOUT PIPELINE
text_sequence = TextToSequence(w1 = word_counts_1.value, w2 = word_counts_2.value)
df_example, www2, bb = text_sequence.transform(r2)

AttributeError: 'MultiIndex' object has no attribute 'labels'

In [30]:
df_example.show(3, truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                   

In [47]:
'fbxtynm' in aa

False

In [55]:
aa == bb

False

In [57]:
x = set(aa) - set(bb)

In [58]:
x

{'aaa',
 'aac',
 'abend',
 'acc',
 'acceleration',
 'accellion',
 'accesscontrol',
 'accessibility',
 'accord',
 'aci',
 'activemq',
 'addphp',
 'addressbar',
 'adm',
 'adminadminphp',
 'airlink',
 'aj',
 'alg',
 'algorithmic',
 'amqp',
 'analytical',
 'apic',
 'apk',
 'applycgi',
 'apport',
 'approve',
 'appsuite',
 'arithmetic',
 'arris',
 'articleid',
 'articlephp',
 'artifact',
 'asf',
 'asu',
 'asustor',
 'aterm',
 'atlas',
 'atp',
 'attendee',
 'auditor',
 'authenticity',
 'authoritysystem',
 'autofill',
 'autonomy',
 'aux',
 'awk',
 'awt',
 'ba',
 'bamboo',
 'baseboard',
 'baseencoded',
 'bat',
 'bbs',
 'be',
 'behaviour',
 'belonging',
 'bigtree',
 'billing',
 'bin',
 'bitbucket',
 'blade',
 'ble',
 'bleichenbacher',
 'blender',
 'bmw',
 'bosch',
 'bosh',
 'bounce',
 'bracket',
 'bs',
 'bulk',
 'busybox',
 'bzip',
 'cansecwest',
 'capnetadmin',
 'carriage',
 'carrier',
 'castle',
 'cde',
 'centoswebpanelcom',
 'centum',
 'cfrelease',
 'cgibin',
 'chassis',
 'checkout',
 'chipse

In [107]:
'tactic' in x

False

In [75]:
ff = ['threatmeter' ,'hack' ,'email' ,'system' ,'hacker' ,'reveal','identity', 'tactics'] 
fff = ['android','malware','target','pcs']
ffff = ['cve', 'bound', 'read', 'intelr', 'baseboard', 'management', 'controller', 'firmware', 'may', 'allow', 'unauthenticated', 'user', 'potentially', 'enable', 'information', 'disclosure', 'via', 'network', 'access', 'cgibincvenam', 'ecginamecve']

In [60]:
r = []
for f in ff:
    r.append(www2[f])
r

[1569, 1125, 3086, 11613, 8097, 706, 849]

In [63]:
r = []
for f in fff:
    r.append(www2[f])
r

[11264, 3664, 1984, 120]

In [56]:
www1['threatmeter']

371

In [59]:
y = set(bb) - set(aa) #과거에는 안 중요했지만 현재는 중요

In [70]:
y

{'abusing',
 'according',
 'accused',
 'actions',
 'activities',
 'actors',
 'added',
 'addresses',
 'admits',
 'adoption',
 'advice',
 'advisories',
 'adware',
 'affecting',
 'agencies',
 'agents',
 'ago',
 'aims',
 'aircraft',
 'alleged',
 'allegedly',
 'alliance',
 'allowed',
 'allowing',
 'alphabetsuccess',
 'americans',
 'americas',
 'amid',
 'analysts',
 'analyzing',
 'androidsecuritynews',
 'announcing',
 'annual',
 'anonymity',
 'appears',
 'applesecuritynews',
 'appreciate',
 'arent',
 'arrest',
 'arrests',
 'ars',
 'articles',
 'artificial',
 'ashley',
 'asia',
 'asked',
 'asks',
 'assange',
 'assessment',
 'atms',
 'attacking',
 'attention',
 'au',
 'aug',
 'australian',
 'authorities',
 'automated',
 'aviation',
 'award',
 'aware',
 'backdoors',
 'bag',
 'band',
 'bans',
 'battery',
 'bay',
 'bbc',
 'beat',
 'becomes',
 'becoming',
 'begins',
 'belevgevgeny',
 'beware',
 'bigger',
 'biometric',
 'birthday',
 'bitcoins',
 'biz',
 'blame',
 'blocks',
 'blogs',
 'bloomberg',
 

In [112]:
'cve' in y

False

In [81]:
asd = []
for f in ffff:
    cnt = 0
    for i in www2.values():
        if i >= int(www2[f]):
            cnt += 1
    asd.append(cnt)
asd

[7,
 2059,
 64,
 1246,
 6264,
 97,
 425,
 153,
 83,
 1,
 142,
 8,
 207,
 156,
 27,
 132,
 4,
 71,
 21,
 1056,
 1054]

In [83]:
len([6, 1592, 61, 1029, 4733, 97, 394, 148, 86, 1, 135, 7, 190, 144, 24, 129, 2, 76, 20, 2300, 2300])

21

In [84]:
len(ffff)

21

In [85]:
len([6, 2047, 63, 1242, 96, 424, 152, 82, 1, 141, 7, 206, 155, 26, 131, 3, 70, 20, 1054, 1053])

20

In [33]:
print(www1['threatmeter'])

371


In [34]:
print(www1['hacked'])

242


In [38]:
print(www1['email']+ www1['emails'])

2599


In [27]:
df_example.select('text', 'feature1').show(3, truncate=False)

+--------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                |feature1                                                                                                                                                                                                                                                                                                                          |
+-------------------------------------------------------------------------------------------------------------------

In [38]:
df_example.rdd.getNumPartitions()

NameError: name 'df_example' is not defined

In [39]:
df_example.count()

NameError: name 'df_example' is not defined

In [40]:
r2.count()

99756

In [41]:
glom_list =  r2.rdd.glom().collect()

In [75]:
print(len(glom_list))
summ = 0
for i in glom_list:
    print(len(i))
    summ += len(i)
print(summ)

5
42029
41818
42744
44450
28471
199512


In [39]:
# df_example.select('label','feature1').rdd.map(lambda x: x).reduceByKey(lambda a,b: a+b).toDF(['key','value'])

DataFrame[key: bigint, value: array<bigint>]

In [27]:
glom_list =  df_example.rdd.glom().collect()

In [37]:
print(len(glom_list))
summ = 0
for i in glom_list:
    print(len(i))
    summ += len(i)
print(summ)

NameError: name 'glom_list' is not defined

In [63]:
df_example.show(5)

+--------------------+-----+--------------------+--------------------+
|                text|label|            feature1|            feature2|
+--------------------+-----+--------------------+--------------------+
|threatmeter hacke...|    1|[962, 1255, 3200,...|[1520, 1002, 2743...|
|first android mal...|    1|[454, 58, 447, 31...|[46, 3722, 0, 0, ...|
|adobe fixes six c...|    1|[105, 3326, 3759,...|[1596, 673, 2609,...|
| scienceporn  in ...|    1|[1562, 0, 0, 0, 0...|[509, 0, 0, 0, 0,...|
|riskware hmoyfzb ...|    1|[0, 0, 0, 0, 0, 0...|[0, 0, 0, 0, 0, 0...|
+--------------------+-----+--------------------+--------------------+
only showing top 5 rows



In [30]:
df_example.select('text','feature1','feature2').show(3, truncate=False)

+--------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                |feature1                                                                                                                

In [35]:
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf
list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())
df_with_vectors = df_example.select(
    df_example["label"], 
    list_to_vector_udf(df_example["feature1"]).alias("feature1"),
    list_to_vector_udf(df_example["feature2"]).alias("feature2")
)

NameError: name 'df_example' is not defined

In [36]:
df_with_vectors.rdd.getNumPartitions()

NameError: name 'df_with_vectors' is not defined

In [33]:
df_with_vectors.printSchema()

root
 |-- label: long (nullable = true)
 |-- feature1: vector (nullable = true)
 |-- feature2: vector (nullable = true)



In [26]:
label_str_index = StringIndexer(inputCol='label', outputCol='label_index')
label_df = label_str_index.fit(df_with_vectors).transform(df_with_vectors)

In [35]:
label_df.printSchema()

root
 |-- label: long (nullable = true)
 |-- feature1: vector (nullable = true)
 |-- feature2: vector (nullable = true)
 |-- label_index: double (nullable = false)



In [34]:
glom_list =  label_df.rdd.glom().collect()
print(len(glom_list))
summ = 0
for i in glom_list:
    print(len(i))
    summ += len(i)
print(summ)

306
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163
163


In [35]:
label_df.rdd.getNumPartitions()

306

In [27]:
nb_classes = label_df.select("label").distinct().count()
nb_classes

2

In [28]:
sc = spark._jsc.sc() 

result1 = sc.getExecutorMemoryStatus().keys() # will print all the executors + driver available

result2 = len([executor.host() for executor in sc.statusTracker().getExecutorInfos() ]) -1

print(result1, end ='\n')
print(result2)

Set(kafka2:33577, kafka1:43117, kafka3:42453)
2


In [29]:
from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.layers import Input, Dense, concatenate
from tensorflow.python.keras.layers import LSTM
from tensorflow.python.keras.layers.embeddings import Embedding
from tensorflow.python.keras.layers import Conv1D, Flatten, GlobalMaxPooling1D, Dropout, MaxPooling1D, Activation, Bidirectional, Conv2D, GlobalMaxPooling2D

In [30]:
vocab_sizes = 5001
neg_vocab_sizes = 5001
embedding_vector_length = 100
hidden_dims = 250
max_length = 100

In [31]:
from tensorflow.keras.callbacks import ModelCheckpoint

model = Sequential()
model.add(Embedding(vocab_sizes, embedding_vector_length, input_length=max_length))

#어휘 크기(최대 정수 인덱스+1, 고밀도 임베딩의 치수, 입력 시퀀스의 길이

# 모델은 크기의 정수 행렬 (배치,
# input_length) 및 입력에서 가장 큰 정수 (즉, 단어 인덱스)
#은 999 (어휘 크기)보다 크지 않아야합니다.
# 이제 model.output_shape는 (None, 10, 64)이고, 여기서`None`은 배치입니다.
# dimension.


# model.add(Conv1D(128, 5, activation='relu'))
model.add(Bidirectional(LSTM(128)))
# model.add(GlobalMaxPooling1D())
model.add(Activation('relu'))
model.add(Dense(nb_classes))
model.add(Activation('sigmoid'))
# model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# #체크포인트 생성?!
# # checkpoint_filepath = '/root/spark/baseline/'
# fname = "checkpoint-{epoch:02d}-{val_loss:.2f}"
# checkpoint = ModelCheckpoint(fname, monitor="val_loss", mode="min",
#     save_best_only=True, verbose=1)
# callbacks = [checkpoint]

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 100)          500100    
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               234496    
_________________________________________________________________
activation (Activation)      (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 2)                 514       
_________________________________________________________________
activation_1 (Activation)    (None, 2)                 0         
Total params: 735,110
Trainable params: 735,110
Non-trainable params: 0
_________________________________________________________________
None


In [35]:
from keras import optimizers, regularizers
from tensorflow.keras import optimizers

optimizer_conf = optimizers.Adam(lr=0.01)
opt_conf = optimizers.serialize(optimizer_conf)

In [36]:
from tensorflow.keras.models import model_from_json
from elephas import spark_model, ml_model
from elephas.ml_model import ElephasEstimator

estimator = ElephasEstimator()
estimator.setFeaturesCol("feature1")
estimator.setLabelCol("label_index")
estimator.set_keras_model_config(model.to_json())
estimator.set_categorical_labels(True) # dense 1이면 False로 설정해야함
estimator.set_nb_classes(nb_classes)
estimator.set_num_workers(2)
estimator.set_epochs(10)
estimator.set_batch_size(32)
estimator.set_verbosity(1)
estimator.set_validation_split(0.10)
estimator.set_optimizer_config(opt_conf)
estimator.set_mode("asynchronous")
estimator.set_loss("binary_crossentropy")
estimator.set_metrics(['acc'])

ElephasEstimator_68dcea20e778

In [37]:
dl_pipeline = Pipeline(stages=[estimator])

In [38]:
import time
t1 = time.time()
my_dl = dl_pipeline.fit(label_df)
t2 = time.time() - t1
print(t2)

>>> Fit model
 * Serving Flask app 'elephas.parameter.server' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://172.28.0.1:4000/ (Press CTRL+C to quit)


>>> Initialize workers
>>> Distribute load


172.28.0.3 - - [12/Oct/2021 05:40:40] "GET /parameters HTTP/1.1" 200 -
172.28.0.2 - - [12/Oct/2021 05:40:40] "GET /parameters HTTP/1.1" 200 -
172.28.0.2 - - [12/Oct/2021 05:43:24] "POST /update HTTP/1.1" 200 -
172.28.0.2 - - [12/Oct/2021 05:43:24] "GET /parameters HTTP/1.1" 200 -
172.28.0.3 - - [12/Oct/2021 05:43:26] "POST /update HTTP/1.1" 200 -
172.28.0.3 - - [12/Oct/2021 05:43:26] "GET /parameters HTTP/1.1" 200 -
172.28.0.2 - - [12/Oct/2021 05:45:53] "POST /update HTTP/1.1" 200 -
172.28.0.2 - - [12/Oct/2021 05:45:53] "GET /parameters HTTP/1.1" 200 -
172.28.0.3 - - [12/Oct/2021 05:46:36] "POST /update HTTP/1.1" 200 -
172.28.0.3 - - [12/Oct/2021 05:46:36] "GET /parameters HTTP/1.1" 200 -
172.28.0.2 - - [12/Oct/2021 05:48:21] "POST /update HTTP/1.1" 200 -
172.28.0.2 - - [12/Oct/2021 05:48:21] "GET /parameters HTTP/1.1" 200 -
172.28.0.3 - - [12/Oct/2021 05:49:52] "POST /update HTTP/1.1" 200 -
172.28.0.3 - - [12/Oct/2021 05:49:52] "GET /parameters HTTP/1.1" 200 -
172.28.0.2 - - [12/Oct/2

>>> Async training complete.
1803.3999667167664


In [43]:
print(t2)

2137.4992032051086


In [ ]:
###################################################################################

In [116]:
csi_pos_neg.show(3, truncate=False)

+--------------------------------------------------------------------------------------------------------------------+-----+
|text                                                                                                                |label|
+--------------------------------------------------------------------------------------------------------------------+-----+
|threatmeter hacked emails of san francisco muni rail system hacker reveals clues about identity and tactics kr  fin |1    |
|first android malware targeting pcs uncovered  fbxtynm                                                              |1    |
|adobe fixes six code execution bugs in flash  mokgxr                                                                |1    |
+--------------------------------------------------------------------------------------------------------------------+-----+
only showing top 3 rows



In [101]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(label_dft.toPandas()['feature1'].to_numpy(), label_dft.toPandas()['label_index'].to_numpy(), test_size=0.2, random_state=42)

In [106]:
from hyperopt import STATUS_OK
from hyperas.distributions import choice, uniform

def data():
    x_train, y_train, x_test, y_test = train_test_split(label_dft.toPandas()['feature1'].to_numpy(), label_dft.toPandas()['label_index'].to_numpy(), test_size=0.2, random_state=42)
    return x_train, y_train, x_test, y_test
#     from tensorflow.keras.datasets import mnist
#     from tensorflow.keras.utils import to_categorical
#     (x_train, y_train), (x_test, y_test) = mnist.load_data()
#     x_train = x_train.reshape(60000, 784)
#     x_test = x_test.reshape(10000, 784)
#     x_train = x_train.astype('float32')
#     x_test = x_test.astype('float32')
#     x_train /= 255
#     x_test /= 255
#     nb_classes = 10
#     y_train = to_categorical(y_train, nb_classes)
#     y_test = to_categorical(y_test, nb_classes)
#     return x_train, y_train, x_test, y_test


def model(x_train, y_train, x_test, y_test):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, Dropout, Activation
    from tensorflow.keras.optimizers import RMSprop

    model = Sequential()
    model.add(Embedding(vocab_sizes, embedding_vector_length, input_length=max_length))
    # model.add(Conv1D(128, 5, activation='relu'))
    model.add(Bidirectional(LSTM(128)))
    # model.add(GlobalMaxPooling1D())
    model.add(Activation('relu'))
    model.add(Dense(nb_classes))
    model.add(Activation('sigmoid'))
    # model.add(Dense(1, activation='sigmoid'))


#     rms = RMSprop()
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#     model.compile(loss='categorical_crossentropy', optimizer=rms)

    model.fit(x_train, y_train,
              batch_size={{choice([64, 128])}},
              nb_epoch=1,
              show_accuracy=True,
              verbose=2,
              validation_data=(x_test, y_test))
    score, acc = model.evaluate(x_test, y_test, show_accuracy=True, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model.to_json()}

In [107]:
from elephas.hyperparam import HyperParamModel
from pyspark import SparkContext, SparkConf

# Define hyper-parameter model and run optimization
hyperparam_model = HyperParamModel(sc)

In [110]:
hyperparam_model.minimize(model=model, data=data, max_evals=5)

FileNotFoundError: [Errno 2] No such file or directory: '/root/spark-2.4.7-bin-hadoop2.7/<ipython-input-110-781bd3e6a9b8>'

In [ ]:
result = hyperparam_model.best_ensemble(nb_ensemble_models=10, model=model, data=train_test_split(label_dft.toPandas()['feature1'].to_numpy(), label_dft.toPandas()['label_index'].to_numpy(), test_size=0.2, random_state=42), max_evals=5)

In [ ]:
###################################################################################

In [58]:
fourth_test.count()

249390

In [45]:
# get_tokens 함수과 같은 용도
stage_1 = RegexTokenizer(inputCol= 'text', outputCol='pos_t', pattern= '\\W')

In [46]:
rt1 = stage_1.transform(first_test)
rt1.show(5)

+---+--------------------+-----+--------------------+
|_c0|                text|label|               pos_t|
+---+--------------------+-----+--------------------+
|  0|threatmeter hacke...|    1|[threatmeter, hac...|
|  1|first android mal...|    1|[first, android, ...|
|  2|adobe fixes six c...|    1|[adobe, fixes, si...|
|  3| scienceporn  in ...|    1|[scienceporn, in,...|
|  4|riskware hmoyfzb ...|    1|[riskware, hmoyfz...|
+---+--------------------+-----+--------------------+
only showing top 5 rows



In [47]:
# get_lemma 함수와 같은 용도
stage_2 = StopWordsRemover(inputCol= stage_1.getOutputCol(), outputCol= 'filtered_words')

In [48]:
rt2 = stage_2.transform(rt1)
rt2.show(5)

+---+--------------------+-----+--------------------+--------------------+
|_c0|                text|label|               pos_t|      filtered_words|
+---+--------------------+-----+--------------------+--------------------+
|  0|threatmeter hacke...|    1|[threatmeter, hac...|[threatmeter, hac...|
|  1|first android mal...|    1|[first, android, ...|[first, android, ...|
|  2|adobe fixes six c...|    1|[adobe, fixes, si...|[adobe, fixes, si...|
|  3| scienceporn  in ...|    1|[scienceporn, in,...|[scienceporn, vac...|
|  4|riskware hmoyfzb ...|    1|[riskware, hmoyfz...|[riskware, hmoyfz...|
+---+--------------------+-----+--------------------+--------------------+
only showing top 5 rows



In [49]:
rt2.rdd.getNumPartitions()

400

In [50]:
rt2.count()

99756

In [69]:
class BroadcastWrapper(object):
    def __init__(self, data, token_list):
        self.broadcast_var = spark.sparkContext.broadcast(data)
#         self.last_updated_time = datetime.now()
        self.token_list = token_list
    
#     def is_should_be_updated(self, data):
#         cur_time = datetime.now()
#         diff_sec = (cur_time - self.last_updated_time).total_seconds()
#         return self.broadcast_var is None or diff_sec> 1
    
    def update_and_get_data(self, spark):
        a = self.broadcast_var.value
        self.broadcast_var.unpersist()
        for i in self.token_list:
            for j in i:
                if j not in a.keys():
                    a[j] = 1
                else:
                    a[j] += 1
        new_data = a
        self.broadcast_var = spark.broadcast(new_data)
#         self.last_updated_time = datetime.now()
#         return len(self.token_list)
        return self.broadcast_var

# TEST용

In [74]:
from pyspark.ml import Transformer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
from typing import Dict
from pyspark.sql import DataFrame

class TextToSequence(Transformer):
    w1 = dict()
    w2 = dict()
    pos_t = Tokenizer()
    neg_t = Tokenizer()
    pos_vocab = []
    neg_vocab = []
    
    def __init__(self, w1: Dict[str, int], w2:Dict[str, int]):
        super(TextToSequence, self).__init__()
        self.w1 = w1
        self.w2 = w2
        
    
    def _transform(self, df: DataFrame):
        result_pdf = df.select("filtered_words").toPandas()[:49878]
        pos_df = result_pdf[:24939] # 
        neg_df = result_pdf[24939:]
        
        # csi_pos인 filtered_words에 대해서만...
        broadcast_wrapper1 = BroadcastWrapper(self.w1, pos_df["filtered_words"])
        ww1 = broadcast_wrapper1.update_and_get_data(spark.sparkContext).value
        
        broadcast_wrapper2 = BroadcastWrapper(self.w2, neg_df["filtered_words"])
        ww2 = broadcast_wrapper2.update_and_get_data(spark.sparkContext).value
        
        www1 = {k: v for k, v in sorted(ww1.items(), key=lambda item: item[1], reverse=True)}
        www2 = {k: v for k, v in sorted(ww2.items(), key=lambda item: item[1], reverse=True)}
        print(www1['allow'])
        print(www2['like'])
        aa = [w for i, w in enumerate(www1) if i < 5000]
        bb = [w for i, w in enumerate(www2) if i < 5000]
        
        self.pos_t.fit_on_texts(aa)
        self.neg_t.fit_on_texts(bb)

#         encoded_docs_pos = self.pos_t.texts_to_sequences(df.select("filtered_words1").toPandas())
#         encoded_docs_neg = self.neg_t.texts_to_sequences(df.select("filtered_words1").toPandas())
        encoded_docs_pos = self.pos_t.texts_to_sequences(result_pdf["filtered_words"])
        encoded_docs_neg = self.neg_t.texts_to_sequences(result_pdf["filtered_words"])

        X_p = pad_sequences(encoded_docs_pos, maxlen=100, padding='post').tolist()
        X_n = pad_sequences(encoded_docs_neg, maxlen=100, padding='post').tolist()
        
        text_array = [str(row['text']) for row in df.select('text').collect()[:49878]]
        label_array = [int(row['label']) for row in df.select('label').collect()[:49878]]

        
        zip_array = list(zip(text_array, label_array, X_p, X_n))
        rdd = spark.sparkContext.parallelize(zip_array, numSlices=306)
        fdf = rdd.toDF(['text','label','feature1','feature2'])
        
        return fdf

In [75]:
# USE THE TRANSFORMER WITHOUT PIPELINE
text_sequence = TextToSequence(w1 = word_counts_1.value, w2 = word_counts_2.value)
dft_example = text_sequence.transform(rt2)

105851
83789


In [76]:
dft_example.count() # 49878

49878

In [77]:
dft_example.select('feature1','feature2').show(3, truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|feature1                                                                                                                                                                                                                                                                                                                          |feature2                                 

In [78]:
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf
list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())
dft_with_vectors = dft_example.select(
    dft_example["label"], 
    list_to_vector_udf(dft_example["feature1"]).alias("feature1"),
    list_to_vector_udf(dft_example["feature2"]).alias("feature2")
)

In [79]:
dft_with_vectors.rdd.getNumPartitions()

306

In [80]:
dft_with_vectors.printSchema()

root
 |-- label: long (nullable = true)
 |-- feature1: vector (nullable = true)
 |-- feature2: vector (nullable = true)



In [81]:
label_str_index = StringIndexer(inputCol='label', outputCol='label_index')
label_dft = label_str_index.fit(dft_with_vectors).transform(dft_with_vectors)

In [64]:
label_dft.printSchema()

root
 |-- label: string (nullable = true)
 |-- feature1: vector (nullable = true)
 |-- feature2: vector (nullable = true)
 |-- label_index: double (nullable = false)



In [52]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics

In [53]:
# pred_train = my_dl.transform(d1)
pred_test = my_dl.transform(label_dft)

NameError: name 'my_dl' is not defined

In [84]:
pred_test.show(5)

+-----+--------------------+--------------------+-----------+--------------------+
|label|            feature1|            feature2|label_index|          prediction|
+-----+--------------------+--------------------+-----------+--------------------+
|    1|[962.0,1255.0,320...|[1520.0,1002.0,27...|        1.0|[1.01370033007697...|
|    1|[454.0,58.0,447.0...|[46.0,3722.0,0.0,...|        1.0|[3.47197055816650...|
|    1|[105.0,3326.0,375...|[1596.0,673.0,260...|        1.0|[2.83003537333570...|
|    1|[1562.0,0.0,0.0,0...|[509.0,0.0,0.0,0....|        1.0|[0.67849528789520...|
|    1|[0.0,0.0,0.0,0.0,...|[0.0,0.0,0.0,0.0,...|        1.0|[0.38029521703720...|
+-----+--------------------+--------------------+-----------+--------------------+
only showing top 5 rows



In [85]:
from pyspark.sql.functions import udf
list_to_double = udf(lambda l: l[0], DoubleType())
pred_test = pred_test.select(
    pred_test["label_index"], 
    list_to_double(pred_test["prediction"]).alias("prediction2")
)

In [86]:
pred_test.printSchema()

root
 |-- label_index: double (nullable = false)
 |-- prediction2: double (nullable = true)



In [87]:
pred_test.show(10, truncate=False)

+-----------+---------------------+
|label_index|prediction2          |
+-----------+---------------------+
|1.0        |1.013700330076972E-5 |
|1.0        |3.471970558166504E-4 |
|1.0        |2.8300353733357042E-5|
|1.0        |0.6784952878952026   |
|1.0        |0.3802952170372009   |
|1.0        |1.2230085849296302E-4|
|1.0        |4.285579780116677E-5 |
|1.0        |1.5866756439208984E-4|
|1.0        |0.3802952170372009   |
|1.0        |6.310045719146729E-4 |
+-----------+---------------------+
only showing top 10 rows



In [88]:
from pyspark.sql.functions import udf
list_to_vector_udf = udf(lambda l:1.0 if l<0.5 else 0.0 , DoubleType())
pred_test = pred_test.select(
    pred_test["label_index"], 
    list_to_vector_udf(pred_test["prediction2"]).alias("rawPrediction")
)

In [89]:
pred_test.show(10, truncate=False)

+-----------+-------------+
|label_index|rawPrediction|
+-----------+-------------+
|1.0        |1.0          |
|1.0        |1.0          |
|1.0        |1.0          |
|1.0        |0.0          |
|1.0        |1.0          |
|1.0        |1.0          |
|1.0        |1.0          |
|1.0        |1.0          |
|1.0        |1.0          |
|1.0        |1.0          |
+-----------+-------------+
only showing top 10 rows



In [90]:
pred_test.groupBy('rawPrediction').count().orderBy('count').show()

KeyboardInterrupt: 

In [69]:
from pyspark.sql.functions import isnan, when, count, col

pred_test.select([count(when(isnan(c), c)).alias(c) for c in pred_test.columns]).show()

+-----------+-------------+
|label_index|rawPrediction|
+-----------+-------------+
|          0|            0|
+-----------+-------------+



#### ROC AUC구하기

In [91]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Let's use the run-of-the-mill evaluator
evaluator = BinaryClassificationEvaluator(labelCol='label_index')

# We have only two choices: area under ROC and PR curves :-(
auroc = evaluator.evaluate(pred_test, {evaluator.metricName: "areaUnderROC"})
auprc = evaluator.evaluate(pred_test, {evaluator.metricName: "areaUnderPR"})
print("Area under ROC Curve: {:.4f}".format(auroc))
print("Area under PR Curve: {:.4f}".format(auprc))

Py4JJavaError: An error occurred while calling o862.evaluate.
: org.apache.spark.SparkException: Job 43 cancelled because killed via the Web UI
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1925)
	at org.apache.spark.scheduler.DAGScheduler.handleJobCancellation(DAGScheduler.scala:1860)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleStageCancellation$1.apply$mcVI$sp(DAGScheduler.scala:1849)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleStageCancellation$1.apply(DAGScheduler.scala:1842)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleStageCancellation$1.apply(DAGScheduler.scala:1842)
	at scala.collection.IndexedSeqOptimized$class.foreach(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofInt.foreach(ArrayOps.scala:234)
	at org.apache.spark.scheduler.DAGScheduler.handleStageCancellation(DAGScheduler.scala:1842)
	at org.apache.spark.scheduler.DAGSched

#### Precision, Recall 구하기

In [74]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="label_index", predictionCol="rawPrediction", metricName="accuracy")

In [75]:
accuracy = evaluator.evaluate(pred_test)
print(accuracy)

0.8850595452905088


In [76]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="label_index", predictionCol="rawPrediction", metricName="f1")

In [77]:
f1 = evaluator.evaluate(pred_test)
print(f1)

0.8843462839707513


In [112]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="label_index", predictionCol="rawPrediction", metricName="weightedPrecision")

In [113]:
weightedPrecision = evaluator.evaluate(pred_test)
print(weightedPrecision)

0.9094533993401972


In [114]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="label_index", predictionCol="rawPrediction", metricName="weightedRecall")

In [115]:
weightedRecall = evaluator.evaluate(pred_test)
print(weightedRecall)

0.9093588355587634


In [72]:
# pnl_train = pred_train.select('label_index', "prediction2")
pnl_test = pred_test.select('label_index', "rawPrediction")

In [73]:
# pred_and_label_train = pnl_train.rdd.map(lambda row: (row['label_index'], row['prediction2']))
pred_and_label_test = pnl_test.rdd.map(lambda row: (row['label_index'],row['rawPrediction']))

In [74]:
# metrics_train = MulticlassMetrics(pred_and_label_train)
metrics_test = MulticlassMetrics(pred_and_label_test)

In [75]:
# print("Training Data Accuracy: {}".format(round(metrics_train.precision(), 4)))
# print("Training Data Confusion Matrix")
# display(pnl_train.crosstab('label_index', 'prediction').toPandas())

print("\nTest Data Accuracy: {}".format(metrics_test.precision()))
# print("Test Data Confusion Matrix")
# display(pnl_test.crosstab("label_index", "prediction").toPandas())

TypeError: Invalid argument, not a string or column: 0.9539476322226232 of type <class 'float'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.

In [76]:
metrics_test.precision()

0.9539476322226232

In [29]:
word_counts_1.value.get('allow')

105727

#### Broadcast 변수 update

In [63]:
class BroadcastWrapper(object):
    def __init__(self, data, token_list):
        self.broadcast_var = data
#         self.last_updated_time = datetime.now()
        self.token_list = token_list
        
#     def is_should_be_updated(self, data):
#         cur_time = datetime.now()
#         diff_sec = (cur_time - self.last_updated_time).total_seconds()
#         return self.broadcast_var is None or diff_sec> 1
    
    def update_and_get_data(self, spark):
        a = self.broadcast_var.value
        self.broadcast_var.unpersist()
        for i in self.token_list:
            a[i] += 1
        new_data = a
        
        self.broadcast_var = spark.broadcast(new_data)
#         self.last_updated_time = datetime.now()
    
        return self.broadcast_var

In [64]:
import datetime
aa = sc.broadcast({'a':1, 'b':2, 'c':3})
broadcast_wrapper = BroadcastWrapper(aa,['a', 'c'] )

In [65]:
a = broadcast_wrapper.update_and_get_data(sc)

In [66]:
a.value

{'a': 2, 'b': 2, 'c': 4}

In [69]:
broadcast_wrapper.broadcast_var.value

{'a': 2, 'b': 2, 'c': 4}

In [73]:
conf = spark.sparkContext._conf.setAll([("spark.streaming.concurrentJobs", "2")])

spark = SparkSession \
            .builder \
            .appName("TwitterClassification") \
            .config(conf=conf) \
            .config("spark.streaming.concurrentJobs", "2") \
            .getOrCreate()

In [74]:
sc.getConf().getAll()

[('spark.eventLog.enabled', 'true'),
 ('spark.driver.memory', '4g'),
 ('spark.executor.id', 'driver'),
 ('spark.history.fs.logDirectory', 'file:///root/spark/eventLog'),
 ('spark.driver.host', 'kafka1'),
 ('spark.network.timeout', '3600s'),
 ('spark.app.name', 'TwitterClassification'),
 ('spark.executor.memory', '6g'),
 ('spark.serializer', 'org.apache.spark.serializer.KryoSerializer'),
 ('spark.streaming.concurrentJobs', '2'),
 ('spark.sql.catalogImplementation', 'hive'),
 ('spark.driver.port', '38277'),
 ('spark.rdd.compress', 'True'),
 ('spark.app.id', 'local-1632550969090'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.default.parallelism', '20'),
 ('spark.submit.deployMode', 'client'),
 ('spark.eventLog.dir', 'file:///root/spark/eventLog'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.kryoserializer.buffer.max', '2047'),
 ('spark.master', 'local[9]')]

In [68]:
spark

In [ ]:
dl_pipeline = Pipeline(stages=[estimator])
model = dl_pipeline.fit(df.map())

In [ ]:
if __name__ == "__main__":
    spark = SparkSession \
            .builder \
            .appName("TwitterClassification") \
            .config("spark.streaming.concurrentJobs", "2")
            .getOrCreate()

    spark.sparkContext.setLogLevel("ERROR")
    
    kafka_df = spark.readStream.format("kafka").option("partition.assignment.strategy","range").option("kafka.bootstrap.servers", "kafka1:9092,kafka2:9092,kafka3:9093").option("subscribe", "test1,test2,test3").option("startingOffsets", "latest").load()
    kafka_df.printSchema()
    kafka_df1 = kafka_df.selectExpr("CAST(value AS STRING)", "timestamp")
    
    transaction_detail_schema = StructType() \
            .add("id", StringType()) \
            .add("text", StringType()) \
            .add("created_at", StringType())
    
    kafka_df2 = kafka_df1 \
            .select(from_json(col("value"), transaction_detail_schema).alias("transaction_detail"), "timestamp")

    kafka_df3 = kafka_df2.select("transaction_detail.*", "timestamp")
 
    udf_blob = udf(apply_blob, StringType())
    
    udf_nlkt = udf(nlkt_analysis, StringType())
    
    new_df = kafka_df3.withColumn("status", udf_nlkt("text"))
    new_df = new_df.drop("text")

    def write_mongo_row(df, epoch_id):
        mongoURL = "mongodb://117.17.189.6:27017/tweet.kafka_tweet"
        df.write.format("com.mongodb.spark.sql.DefaultSource").mode("append").option("uri", mongoURL).save()
        df.show()
        pass
    query = new_df.writeStream.trigger(processingTime='1 seconds').foreachBatch(write_mongo_row).start() 
    print('---------------------------------------------')
    query.awaitTermination()

    spark.stop()


In [147]:
d1f = pd.DataFrame([[[80, 0],[90, 1]],[[70,13],[80,34]]],columns=['aa','nn'])
d1f

,aa,nn
0,"[80, 0]","[90, 1]"
1,"[70, 13]","[80, 34]"


In [148]:
d1f['aa'].values

array([list([80, 0]), list([70, 13])], dtype=object)

In [149]:
cols = ['aa','nn']
def make_original(x) :
    print(x.values)
    tmp = []
    tmp.append(x.values)
    return tmp
d1f.insert(0, 'eee', d1f[cols].apply(lambda row: make_original(row), axis=1))

[list([80, 0]) list([90, 1])]
[list([70, 13]) list([80, 34])]


In [150]:
a = list([80, 0])

In [151]:
type(a)

list

In [152]:
a[0]

80

In [153]:
d1f['eee']

0      [[[80, 0], [90, 1]]]
1    [[[70, 13], [80, 34]]]
Name: eee, dtype: object

In [155]:
d1f['eee'].iloc[0][0][0]

[80, 0]

In [112]:
d1f

,eee,aa,nn
0,"[[[80, 0], [90, 1]]]","[80, 0]","[90, 1]"
1,"[[[70, 13], [80, 34]]]","[70, 13]","[80, 34]"


In [ ]:
################################ EX ############################################

In [106]:
import pyspark.sql.functions as F
from pyspark.sql.functions import arrays_zip
sentenceData = spark.createDataFrame([
    (0.0, "Hi I heard about Spark"),
    (0.0, "I wish Java could use case classes"),
    (1.0, "Logistic regression models are neat")
]

df.show(truncate=False)

+---+-----------------------------------+
|_1 |_2                                 |
+---+-----------------------------------+
|0.0|Hi I heard about Spark             |
|0.0|I wish Java could use case classes |
|1.0|Logistic regression models are neat|
+---+-----------------------------------+



In [107]:
list_data = ['a' ,'b', 'c']

df = df.select(collect_list(struct(F.col("*"))).alias("data"))
df.show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------+
|data                                                                                                                  |
+----------------------------------------------------------------------------------------------------------------------+
|[[0.0, Hi I heard about Spark], [0.0, I wish Java could use case classes], [1.0, Logistic regression models are neat]]|
+----------------------------------------------------------------------------------------------------------------------+



In [108]:
df = df.withColumn("list",F.array([F.lit(i) for i in list_data]))
df.show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------+---------+
|data                                                                                                                  |list     |
+----------------------------------------------------------------------------------------------------------------------+---------+
|[[0.0, I wish Java could use case classes], [1.0, Logistic regression models are neat], [0.0, Hi I heard about Spark]]|[a, b, c]|
+----------------------------------------------------------------------------------------------------------------------+---------+



In [109]:
df = df.select(F.explode(F.arrays_zip(F.col("data"),F.col("list"))).alias("full_data"))
df.show(truncate=False)

+-----------------------------------------------+
|full_data                                      |
+-----------------------------------------------+
|[[0.0, I wish Java could use case classes], a] |
|[[1.0, Logistic regression models are neat], b]|
|[[0.0, Hi I heard about Spark], c]             |
+-----------------------------------------------+



In [110]:
df = df.select(F.col("full_data.data.*"),F.col("full_data.list").alias("col3"))
df.show(truncate=False)

+---+-----------------------------------+----+
|_1 |_2                                 |col3|
+---+-----------------------------------+----+
|0.0|Hi I heard about Spark             |a   |
|0.0|I wish Java could use case classes |b   |
|1.0|Logistic regression models are neat|c   |
+---+-----------------------------------+----+



In [ ]:
#################### 해당 내용 list에도 적용가능한지 확인########################

In [42]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
hashingTF = HashingTF(inputCol="pos_t", outputCol="rawFeatures", numFeatures=100)
featurizedData = hashingTF.transform(r4)
featurizedData.show(5)

+---+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|_c0|                text|label|               pos_t|               neg_t|     filtered_words1|     filtered_words2|         rawFeatures|
+---+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|  0|threatmeter hacke...|    0|[threatmeter, hac...|[threatmeter, hac...|[threatmeter, hac...|[threatmeter, hac...|(100,[7,12,15,20,...|
|  1|first android mal...|    0|[first, android, ...|[first, android, ...|[first, android, ...|[first, android, ...|(100,[3,11,40,69,...|
|  2|adobe fixes six c...|    0|[adobe, fixes, si...|[adobe, fixes, si...|[adobe, fixes, si...|[adobe, fixes, si...|(100,[5,20,40,45,...|
|  3| scienceporn  in ...|    0|[scienceporn, in,...|[scienceporn, in,...|[scienceporn, vac...|[scienceporn, vac...|(100,[29,38,45,70...|
|  4|riskware hmoyfzb ...|    0|[r

In [43]:
featurizedData = featurizedData.toPandas()
featurizedData['rawFeatures'].value_counts()

(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)    169
(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

In [44]:
len(featurizedData['rawFeatures'].iloc[1])

100

In [ ]:
### rawFeatures input으로 사용해서 모델 돌려보기

In [18]:
from pyspark.ml.feature import CountVectorizer
stage_3_a = CountVectorizer(inputCol=stage_2_a.getOutputCol(), outputCol='word1', vocabSize=50, minDF=2.0)
stage_3_b = CountVectorizer(inputCol=stage_2_b.getOutputCol(), outputCol='word2', vocabSize=50, minDF=2.0)

In [19]:
pipeline = Pipeline(stages= [stage_1_a, stage_1_b, stage_2_a, stage_2_b, stage_3_a, stage_3_b])
pipelineFit = pipeline.fit(csi_pos_neg)

In [29]:
df_transform = pipelineFit.transform(csi_pos_neg)
dfdfdf = df_transform.toPandas()

In [30]:
type(dfdfdf)

pandas.core.frame.DataFrame

In [36]:
dfdfdf['word1'].value_counts()

(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)    84912
(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)     5534
(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)     3799
(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [35]:
len(dfdfdf['word1'].iloc[1])

50

In [21]:
from pyspark.ml.feature import CountVectorizer
stage_3_a = CountVectorizer(inputCol=stage_2_a.getOutputCol(), outputCol='word1', vocabSize=100, minDF=2.0)
stage_3_b = CountVectorizer(inputCol=stage_2_b.getOutputCol(), outputCol='word2', vocabSize=100, minDF=2.0)

In [26]:
model = stage_3_a.fit(r4)
r5 = model.transform(r4)
r5.show(5)

+---+--------------------+-----+--------------------+--------------------+--------------------+--------------------+---------------+
|_c0|                text|label|               pos_t|               neg_t|     filtered_words1|     filtered_words2|          word1|
+---+--------------------+-----+--------------------+--------------------+--------------------+--------------------+---------------+
|  0|threatmeter hacke...|    0|[threatmeter, hac...|[threatmeter, hac...|[threatmeter, hac...|[threatmeter, hac...| (50,[4],[1.0])|
|  1|first android mal...|    0|[first, android, ...|[first, android, ...|[first, android, ...|[first, android, ...|(50,[29],[1.0])|
|  2|adobe fixes six c...|    0|[adobe, fixes, si...|[adobe, fixes, si...|[adobe, fixes, si...|[adobe, fixes, si...|     (50,[],[])|
|  3| scienceporn  in ...|    0|[scienceporn, in,...|[scienceporn, in,...|[scienceporn, vac...|[scienceporn, vac...|     (50,[],[])|
|  4|riskware hmoyfzb ...|    0|[riskware, hmoyfz...|[riskware, hmoyf

In [27]:
model2 = stage_3_b.fit(r5)
r6 = model2.transform(r5)
r6.show(5)

+---+--------------------+-----+--------------------+--------------------+--------------------+--------------------+---------------+---------------+
|_c0|                text|label|               pos_t|               neg_t|     filtered_words1|     filtered_words2|          word1|          word2|
+---+--------------------+-----+--------------------+--------------------+--------------------+--------------------+---------------+---------------+
|  0|threatmeter hacke...|    0|[threatmeter, hac...|[threatmeter, hac...|[threatmeter, hac...|[threatmeter, hac...| (50,[4],[1.0])| (50,[4],[1.0])|
|  1|first android mal...|    0|[first, android, ...|[first, android, ...|[first, android, ...|[first, android, ...|(50,[29],[1.0])|(50,[29],[1.0])|
|  2|adobe fixes six c...|    0|[adobe, fixes, si...|[adobe, fixes, si...|[adobe, fixes, si...|[adobe, fixes, si...|     (50,[],[])|     (50,[],[])|
|  3| scienceporn  in ...|    0|[scienceporn, in,...|[scienceporn, in,...|[scienceporn, vac...|[sciencepor

In [28]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols= [stage_3_a.getOutputCol(), stage_3_b.getOutputCol()], outputCol= 'features')

In [34]:
r7 = vectorAssembler.transform(r6)
r7.show(5, truncate=False)

+---+--------------------------------------------------------------------------------------------------------------------+-----+-------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------+---------------+---------------+-----------------------+
|_c0|text                                                                                                                |label|pos_t                                                                                                                                |neg_t                                            

In [ ]:
#########################################################################################

In [37]:
from pyspark.ml.feature import PolynomialExpansion
from pyspark.ml.linalg import Vectors

df = spark.createDataFrame([
    (Vectors.dense([2.0, 1.0]),),
    (Vectors.dense([0.0, 0.0]),),
    (Vectors.dense([3.0, -1.0]),)
], ["features"])

polyExpansion = PolynomialExpansion(degree=3, inputCol="features", outputCol="polyFeatures")
polyDF = polyExpansion.transform(df)

polyDF.show(truncate=False)

+----------+------------------------------------------+
|features  |polyFeatures                              |
+----------+------------------------------------------+
|[2.0,1.0] |[2.0,4.0,8.0,1.0,2.0,4.0,1.0,2.0,1.0]     |
|[0.0,0.0] |[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]     |
|[3.0,-1.0]|[3.0,9.0,27.0,-1.0,-3.0,-9.0,1.0,3.0,-1.0]|
+----------+------------------------------------------+



In [39]:
from pyspark.ml.feature import StringIndexer

df = spark.createDataFrame(
    [(0, "a"), (1, "b"), (2, "c"), (3, "a"), (4, "a"), (5, "c")],
    ["id", "category"])

indexer = StringIndexer(inputCol="category", outputCol="categoryIndex")
indexed = indexer.fit(df).transform(df)
indexed.show()

+---+--------+-------------+
| id|category|categoryIndex|
+---+--------+-------------+
|  0|       a|          0.0|
|  1|       b|          2.0|
|  2|       c|          1.0|
|  3|       a|          0.0|
|  4|       a|          0.0|
|  5|       c|          1.0|
+---+--------+-------------+



In [ ]:
#######################################################################################

In [15]:
pipeline = Pipeline(stages= [stage_1_a, stage_1_b, stage_2_a, stage_2_b, stage_3_a, stage_3_b, vectorAssembler])
pipelineFit = pipeline.fit(csi_pos_neg)

In [16]:
df_transform = pipelineFit.transform(csi_pos_neg)
df_transform.limit(5).toPandas()

,_c0,text,label,pos_t,neg_t,filtered_words1,filtered_words2,word1,word2,features
0,0,threatmeter hacked emails of san francisco mun...,0,"[threatmeter, hacked, emails, of, san, francis...","[threatmeter, hacked, emails, of, san, francis...","[threatmeter, hacked, emails, san, francisco, ...","[threatmeter, hacked, emails, san, francisco, ...","(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1,first android malware targeting pcs uncovered ...,0,"[first, android, malware, targeting, pcs, unco...","[first, android, malware, targeting, pcs, unco...","[first, android, malware, targeting, pcs, unco...","[first, android, malware, targeting, pcs, unco...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,2,adobe fixes six code execution bugs in flash ...,0,"[adobe, fixes, six, code, execution, bugs, in,...","[adobe, fixes, six, code, execution, bugs, in,...","[adobe, fixes, six, code, execution, bugs, fla...","[adobe, fixes, six, code, execution, bugs, fla...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,3,scienceporn in a vacuum i guess,0,"[scienceporn, in, a, vacuum, i, guess]","[scienceporn, in, a, vacuum, i, guess]","[scienceporn, vacuum, guess]","[scienceporn, vacuum, guess]","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,4,riskware hmoyfzb fbjaz,0,"[riskware, hmoyfzb, fbjaz]","[riskware, hmoyfzb, fbjaz]","[riskware, hmoyfzb, fbjaz]","[riskware, hmoyfzb, fbjaz]","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [17]:
df_transform_fin = df_transform.orderBy(rand())
nb_classes = df_transform_fin.distinct().count()

In [18]:
vocab_sizes = 5001
neg_vocab_sizes = 5001
embedding_vector_length = 100
hidden_dims = 250
max_length = 100

In [19]:
from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.layers import Input, Dense, concatenate
from tensorflow.python.keras.layers import LSTM
from tensorflow.python.keras.layers.embeddings import Embedding
from tensorflow.python.keras.layers import Conv1D, Flatten, GlobalMaxPooling1D, Dropout, MaxPooling1D, Activation, Bidirectional, Conv2D, GlobalMaxPooling2D

In [20]:
# CSI positive Embedding model
inputA = Input(shape=(100,))
x = Embedding(vocab_sizes, embedding_vector_length, input_length=max_length)(inputA)
# x = Conv1D(128, 5, activation='relu')(x)
# x = MaxPool1D(strides=1, padding='valid')(x)
x = Bidirectional(LSTM(128))(x)
x = Dropout(0.2)(x)
x = Activation('relu')(x)
x = Model(inputs=inputA, outputs=x)

# CSI negative Embedding model
inputC = Input(shape=(100,))
k = Embedding(neg_vocab_sizes, embedding_vector_length, input_length=max_length)(inputC)
# k = Conv1D(128, 5, activation='relu')(k)
# k = MaxPool1D(strides=1, padding='valid')(k)
k = Bidirectional(LSTM(128))(k)
k = Dropout(0.2)(k)
k = Activation('relu')(k)
k = Model(inputs=inputC, outputs=k)

combined = concatenate([x.output, k.output], axis=1)
# z = Flatten()(combined)
z = Dense(8, activation='relu')(combined)
z = Dense(1, activation="sigmoid")(z)

model = Model(inputs=[x.input, k.input], outputs=z)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [21]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 100)     500100      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 100)     500100      input_2[0][0]                    
____________________________________________________________________________________________

In [22]:
from keras import optimizers, regularizers
from tensorflow.keras import optimizers

optimizer_conf = optimizers.Adam(lr=0.01)
opt_conf = optimizers.serialize(optimizer_conf)

In [41]:
from tensorflow.keras.models import model_from_json
from elephas import spark_model, ml_model
from elephas.ml_model import ElephasEstimator

estimator = ElephasEstimator()
estimator.setFeaturesCol("features")
estimator.setLabelCol("label")
estimator.set_keras_model_config(model.to_json())
# estimator.set_categorical_labels(True)
estimator.set_nb_classes(nb_classes)
estimator.set_num_workers(2)
estimator.set_epochs(3)
estimator.set_batch_size(32)
estimator.set_verbosity(1)
estimator.set_validation_split(0.10)
estimator.set_optimizer_config(opt_conf)
estimator.set_mode("synchronous")
estimator.set_loss("binary_crossentropy")
# estimator.set_metrics(['acc'])

dl_pipeline = Pipeline(stages=[estimator])
my_dl = dl_pipeline.fit(df_transform_fin)

ElephasEstimator_907d50e28eff

In [24]:
dl_pipeline = Pipeline(stages=[estimator])

In [25]:
my_dl = dl_pipeline.fit(df_transform_fin)

>>> Fit model


KeyboardInterrupt: 

In [ ]:
my_dl.write().overwrite().save('hdfs:///user/root/model_2')